In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.signal import argrelmax
from sklearn.preprocessing import RobustScaler
import scipy.signal
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation
from sklearn.metrics import mean_absolute_error
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Input, Conv2D,Conv1D,BatchNormalization, Multiply,Attention, MaxPooling2D, Bidirectional,  Flatten, Concatenate, Dot, Softmax, Dense, LSTM, Dropout, concatenate,BatchNormalization,Flatten, Reshape
from tensorflow.keras.initializers import RandomNormal
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input
from scipy.signal import spectrogram
from tensorflow.keras import initializers
from PIL import Image
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import csv
from sklearn.ensemble import RandomForestRegressor
import math
import pickle
from ctgan import CTGAN
import xgboost as xgb
from scipy import integrate
from sklearn.metrics import r2_score
from scipy import signal
import scipy
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import load_model
from scipy.integrate import quad
from tensorflow.keras.layers import GRU, Dense, Dropout
from math import cos, asin, sqrt, pi
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.initializers import he_normal
from scipy.fft import fft, fftfreq
# importing the modules
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Input, LSTM, Dense, Dropout, Reshape, Flatten, concatenate, Dot, Concatenate, Lambda
import keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from matplotlib.mlab import specgram
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
sns.set_style('whitegrid')
sns.set_context("notebook")
sns.set_palette("husl")
import numpy as np
from tensorflow.keras.initializers import RandomNormal
from sklearn.model_selection import train_test_split
from keras_self_attention import SeqSelfAttention
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Dense, Dot, Concatenate, RepeatVector
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D,BatchNormalization, Attention, MaxPooling2D, Bidirectional,  Flatten, Concatenate, Dot, Softmax, Dense, LSTM, Dropout, concatenate,BatchNormalization,Flatten, Reshape
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import pickle as pkl
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

print(tf.__version__)

print(tf.config.list_physical_devices('GPU'))

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
print("X train size:",tab_train)

In [ ]:
# MY MAIN MODEL
from tensorflow.signal import fft


with tf.device('/GPU:0'):
    def create_attention_cnn_model(your_sequence_length, your_num_features):
        input_layer = Input(shape=(your_sequence_length, your_num_features))
        lstm_layer = LSTM(64,return_sequences=True)(input_layer)
        lstm_layer = LSTM(128,return_sequences=True)(lstm_layer)
        
        flattened_representation = Flatten()(lstm_layer)
        tabular_input = Input(shape=(16,))
        concat = tf.keras.layers.concatenate([flattened_representation, tabular_input])
        flatten_layer = Flatten()(concat)
        dense1 = Dense(64, activation='relu')(flatten_layer)
        dense1 = Dense(32, activation='relu')(dense1)
        dense1 = Dense(16, activation='relu')(dense1)
        output_layer = Dense(1)(dense1)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.001,
                decay_steps=100,
                decay_rate=0.96,
                staircase=True
            )
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        model_cnn_bilstm = Model(inputs=[input_layer, tabular_input], outputs=output_layer)
        model_cnn_bilstm.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        return model_cnn_bilstm

    your_sequence_length = 300 
    your_num_features = 5
    model = create_attention_cnn_model(your_sequence_length, your_num_features)
    checkpoint = ModelCheckpoint(filepath='Attention_weights.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
    model.fit(x=[seismic_train,tab_train], y=y_train, batch_size=32, epochs=10, 
              validation_data=([seismic_val,tab_val], y_val),callbacks=[checkpoint,early_stopping])
    model.load_weights('Attention_weights.hdf5')



In [ ]:
y_pred = model.predict([seismic_test,tab_test])
y_pred_test=y_pred
mae = mean_absolute_error(y_test,y_pred_test)
print("Mean Absolute Error:", mae)
count=0
for i in range(0,y_test.shape[0]):
    if np.abs(y_test[i])>=100:
        if np.abs(y_pred[i])>=100:
            count=count+1
print(count)
print(y_test.shape)
plt.scatter(y_test,y_pred_test,s=10,color='black')

In [ ]:

def save_ctgan(tab_train,y_pred_train,y_train):
    combined_data = np.concatenate((tab_train,y_pred_train), axis=1)
    
    y_train_reshaped = tf.reshape(y_train, (-1, 1))
    data = np.concatenate((combined_data,y_train_reshaped), axis=1)
    print(data.shape)
    ctgan = CTGAN(epochs=100)
    print("Creating new data")
    ctgan.fit(data)
    print("FITTED THE CTGAN mdoel")
    model_file = "Model New/ctgan_Magnitude_Pred.pkl"
    with open(model_file, 'wb') as model_filename:
        pickle.dump(ctgan, model_filename)
    return ctgan

model = load_model('Model New/Model_Magnitude_Predict.h5')

y_pred_train = model.predict([seismic_train,tab_train])    
ctgan_model_HIGHER=save_ctgan(tab_train,y_pred_train,y_train)

In [ ]:
combined_data = np.concatenate((tab_train,y_pred_train), axis=1)
y_train_reshaped = tf.reshape(y_train, (-1, 1))
data = np.concatenate((combined_data,y_train_reshaped), axis=1)

In [ ]:
dense_output = model.predict([ seismic_test,tab_test]) 
combined_data = np.concatenate((tab_test, dense_output), axis=1)
y_pred_train = model.predict([seismic_train,tab_train])    

In [ ]:
combined_data2 = np.concatenate((tab_train,y_pred_train), axis=1)
y_train_reshaped = tf.reshape(y_train, (-1, 1))
data = np.concatenate((combined_data2,y_train_reshaped), axis=1)
model_file = "Model New/ctgan_Magnitude_Pred.pkl"
with open(model_file, 'rb') as file:
    ctgan_model_HIGHER = pickle.load(file)


In [ ]:
def loading_data_high(synthetic_data,max_size):
    selected_vectors=[]

    sel6=(synthetic_data[np.round(synthetic_data[:, -1],1)==7.5])[:500]
    sel7=(synthetic_data[np.round(synthetic_data[:, -1],1)==7.6])[:500]
    sel8=(synthetic_data[np.round(synthetic_data[:, -1],1)==7.7])[:500]
    sel9=(synthetic_data[np.round(synthetic_data[:, -1],1)==7.8])[:500]
    sel10=(synthetic_data[np.round(synthetic_data[:, -1],1)==7.9])[:500]
    sel11=(synthetic_data[np.round(synthetic_data[:, -1],1)==8.0])[:500]

    data = np.concatenate((sel6,sel7), axis=0)
    data = np.concatenate((data,sel8), axis=0)
    data = np.concatenate((data,sel9), axis=0)
    data = np.concatenate((data,sel10), axis=0)
    data = np.concatenate((data,sel11), axis=0)
    print("Synthetic dataset size",data.shape)
    return data

    
def call_ctgan(ctgan,flag,sample):
    # Generate synthetic data
    num_samples = sample  # Specify the number of synthetic samples you want to generate
    synthetic_data = ctgan.sample(num_samples)

    mask = (synthetic_data >= -1).all(axis=1)
    # Apply the mask to filter the rows
    filtered_synthetic_data = synthetic_data[mask]
    filtered_synthetic_data[:, -1] = np.round(filtered_synthetic_data[:, -1], 1)

    if flag==1:
        synthetic_data=loading_data_high(filtered_synthetic_data,25)   
    return synthetic_data
    
def train_ML(tab_train,y_pred_train,y_train,ctgan_model_HIGHER,data,combined_data):
    synthetic_data=call_ctgan(ctgan_model_HIGHER,1,500000)
    # synthetic_data = np.concatenate((syn_high,syn_low), axis=0)
    print("Before Adding synthetic data",data.shape)
    new_data = np.concatenate((data,synthetic_data), axis=0)
    df = pd.DataFrame(new_data)
    print("After adding synthetic data",new_data.shape)
#     # Specify the CSV file path
    csv_file = "D:/Paper IJCAI/Output Dataset/Pred_Magnitude_Train_Synthetic_Mix_Data_2.csv"
    # Save the DataFrame to a CSV file without column names
    df.to_csv(csv_file, index=False, header=False)
    df_1 = pd.DataFrame(synthetic_data)
    print("After adding synthetic data",data.shape)
#     # Specify the CSV file path
    csv_file = "D:/Paper IJCAI/Output Dataset/Pred_Magnitude_Train_Synthetic_Data.csv"
    # Save the DataFrame to a CSV file without column names
    df_1.to_csv(csv_file, index=False, header=False)
    
    new_data=pd.read_csv(csv_file,sep=',')
    dataset=new_data.values
    t=dataset.shape[1]-1
    X_train_new = dataset[:,0:t]
    y_train_new = dataset[:,t]

train_ML(tab_train,y_pred_train,y_train,ctgan_model_HIGHER,data,combined_data)  

In [ ]:

def evaluate_error(pga_test,predictions):
    predictions=np.round(predictions,decimals=2)
    print("Size is: ",predictions.shape)
    mae = mean_absolute_error(pga_test,predictions)
    print("Mean Absolute Error:", mae)
    diff=np.abs(pga_test- predictions)
    std_dev_pga = np.std(diff)
    print("Standard deviation is",std_dev_pga)
    rmse = np.sqrt(mean_squared_error(pga_test, predictions))
    pred_error=pga_test-predictions
    print("Root Mean Squared Error:", rmse)
    plt.scatter(pga_test,predictions,s=10,color='black') 

def evaluate_error_channel(pga_test, waveform_test, tabular_test,model,
                           LGBM_model,combined_data):
    print("Shape",combined_data.shape)
    predictions=LGBM_model.predict(combined_data)
    predictions=np.round(predictions,decimals=2)
    print("Size is: ",predictions.shape)
    print("y_test",pga_test.shape)
    mae = mean_absolute_error(pga_test,predictions)
    print("Mean Absolute Error:", mae)
    diff=np.abs(pga_test- predictions)
    std_dev_pga = np.std(diff)
    print("Standard deviation is",std_dev_pga)
    rmse = np.sqrt(mean_squared_error(pga_test, predictions))
    pred_error=pga_test-predictions
    print("Root Mean Squared Error:", rmse)
    plt.scatter(pga_test,predictions,s=10,color='black')
    plt.xlim(2, 500)
    plt.ylim(2, 500)    
    return predictions
    
def Model_ML(tab_train,y_pred_train,y_train,y_test,seismic_test,tab_test,combined_data,dense_output):   
    csv_file = "D:/Paper IJCAI/Output Dataset/Pred_Magnitude_Train_Synthetic_Mix_Data_2.csv"
    new_data=pd.read_csv(csv_file,sep=',')
    dataset=new_data.values
    t=dataset.shape[1]-1
    X_train_new = dataset[:,0:t]
    y_train_new = dataset[:,t]
    print(X_train_new.shape)
    XGB_model = xgb.XGBRegressor(max_depth=2,learning_rate=0.1,n_estimators=300,subsample=0.6,seed=1,
                            alpha=0.1,min_child_weight=1,random_state=1,scale_pos_weight = 1,base_score=10,
                            reg_lambda=0.2 )
    CatB_model = CatBoostRegressor(iterations=200,  # Number of trees (boosting rounds)
                         learning_rate=0.01,  # Step size for the optimization
                        depth=6,  # Tree depth
                        subsample=0.7,
                         verbose=100)
    RF1_model = RandomForestRegressor(n_estimators=200,  # Number of trees in the forest
                             max_depth=3,  # Maximum depth of each tree
                             random_state=42)  # Set a random seed for reproducibility  
    LGBM_model = lgb.LGBMRegressor(n_estimators=200,learning_rate=0.15,max_depth=6,random_state=42)  # Set a random seed for reproducibility
    XGB_model_Real = xgb.XGBRegressor(max_depth=4,learning_rate=0.1,n_estimators=100,subsample=0.6,seed=1,
                            alpha=0.1,min_child_weight=1,random_state=1,scale_pos_weight = 1,base_score=10,
                            reg_lambda=0.2 )
    CatB_model_Real = CatBoostRegressor(iterations=100,  # Number of trees (boosting rounds)
                         learning_rate=0.1,  # Step size for the optimization
                        depth=6,  # Tree depth
                        subsample=0.7,
                         verbose=100)
    RF1_model_real = RandomForestRegressor(n_estimators=100,  # Number of trees in the forest
                             max_depth=10,  # Maximum depth of each tree
                             random_state=42)  # Set a random seed for reproducibility  
    LGBM_model_Real = lgb.LGBMRegressor(n_estimators=200,learning_rate=0.1,max_depth=10,random_state=42)  # Set a random seed for reproducibility
    dense_output_train = model.predict([ seismic_train,tab_train]) 
    combined_train = np.concatenate((tab_train, dense_output_train), axis=1)
    X_train=combined_train
    XGB_model_Real.fit(X_train, y_train)
    XGB_model.fit(X_train_new, y_train_new)
    predictions_XGB=evaluate_error_channel(y_test, seismic_test,tab_test,model,XGB_model,combined_data)
    predictions_XGB_Real=evaluate_error_channel(y_test, seismic_test,tab_test,model,XGB_model_Real,combined_data)
    y_train = np.array(y_train)
    X_train = np.array(X_train)
    print(dense_output.shape)
    dense_output=dense_output.flatten()
    pred=(predictions_XGB_Real+predictions_XGB)/2

    print("Final is",np.mean(pred))
    plt.scatter(y_test,pred,s=10,color='black')
    plt.xlim(2.5, 9)
    plt.ylim(2.5, 9)    
    pred=np.abs(pred)
    evaluate_error(y_test,pred)
    file_path ='Model New/Pred_Mag_XGB_main_model_AUG.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(XGB_model, file)
    file_path ='Model New/Pred_Mag_CatB_main_model_AUG.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(CatB_model, file)
    file_path ='Model New/Pred_Mag_RF1_main_model_AUG.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(RF1_model, file)
    file_path ='Model New/Pred_Mag_LGBM_main_model_AUG.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(LGBM_model, file)

    file_path ='Model New/Pred_Mag_XGB_main_model_REAL.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(XGB_model_Real, file)
    file_path ='Model New/Pred_Mag_CatB_main_model_REAL.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(CatB_model_Real, file)
    file_path ='Model New/Pred_Mag_RF1_main_model_REAL.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(RF1_model_real, file)
    file_path ='Model New/Pred_Mag_LGBM_main_model_REAL.pkl'
    with open(file_path, 'wb') as file:
        pkl.dump(LGBM_model_Real, file)
        
    return XGB_model,CatB_model,RF1_model,LGBM_model
print("1. CNNAttention-----------")
print("TEST")

y_pred_latest = model.predict([seismic_latest,tab_latest])   
dense_output = model.predict([ seismic_latest,tab_latest]) 
combined_data = np.concatenate((tab_latest, dense_output), axis=1)
XGB_model,CatB_model,RF1_model,LGBM_model=Model_ML(tab_train,y_pred_train,y_train,y_latest,seismic_latest,tab_latest,combined_data,dense_output)